<a href="https://colab.research.google.com/github/Pushkaran-P/Age-Race-Classification/blob/main/Age_FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


NormalTrainingData or EqualDistributionTrainingData

In [ ]:
!unrar x -inul "/content/drive/MyDrive/EqualDistributionTrainingData.rar"
!unrar x -inul "/content/drive/MyDrive/EqualDistributionTestingData.rar"

In [ ]:
import os
import cv2
import numpy as np
from tensorflow import keras

class CustomDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, dir, batch_size=16, n_classes=5):
        'Initialization'
        self.dir = dir
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.on_epoch_end()
        np.random.shuffle(self.indexes)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.list_IDs = []
        self.labels = {}
        for i in range(self.n_classes):
            path = os.path.join(self.dir, str(i))
            for file in os.listdir(path):
                if file.endswith(".jpg"):
                    self.list_IDs.append(os.path.join(path, file))
                    self.labels[os.path.join(path, file)] = i
        self.indexes = np.arange(len(self.list_IDs))
        np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        # Initialization
        X = []
        y = np.empty((self.batch_size), dtype=int)  # ensure y has shape (batch_size)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Load image
            img = cv2.imread(ID)
            X.append(img)

            # Store class
            y[i] = self.labels[ID]  # labels are integers

        # Pad images to max height and width
        max_height = max(img.shape[0] for img in X)
        max_width = max(img.shape[1] for img in X)
        X = [np.pad(img, ((0, max_height - img.shape[0]), (0, max_width - img.shape[1]), (0, 0)), 'constant') for img in X]
        # Convert to numpy array
        X = np.array(X)

        # Normalize images
        X = X / 255.0

        # X = [data_augmentation(img) for img in X]

        return X, y  # y is an array of integer labels


In [ ]:
train_generator = CustomDataGenerator('EqualDistributionTrainingData')
test_generator = CustomDataGenerator('EqualDistributionTestingData')

In [ ]:
import tensorflow as tf

# Detect and initialize GPU
try:
    strategy = tf.distribute.OneDeviceStrategy('GPU')
except tf.errors.NotFoundError:
    print('No GPU devices found. Please make sure GPU is enabled in the runtime settings.')
    strategy = tf.distribute.get_strategy()

In [ ]:
def macro_f1(y_true, y_pred):
    # Convert predicted probabilities to class labels
    y_pred = tf.argmax(y_pred, axis=-1)
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.int32)

    # Calculate the number of true positive, false positive, and false negative predictions for each class
    true_positives = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=0), tf.float32)
    false_positives = tf.cast(tf.math.count_nonzero((1 - y_true) * y_pred, axis=0), tf.float32)
    false_negatives = tf.cast(tf.math.count_nonzero(y_true * (1 - y_pred), axis=0), tf.float32)

    # Calculate precision and recall for each class
    precision = true_positives / (true_positives + false_positives + 1e-6)
    recall = true_positives / (true_positives + false_negatives + 1e-6)

    # Calculate the F1 score for each class
    f1 = 2 * precision * recall / (precision + recall + 1e-6)

    # Calculate the macro-averaged F1 score by taking the mean of the F1 scores for each class
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dense, Activation
from keras.callbacks import EarlyStopping

In [ ]:
# Define your model
model = Sequential()
model.add(layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(None, None, 3)))
model.add(layers.experimental.preprocessing.RandomRotation(0.1))
model.add(layers.experimental.preprocessing.RandomZoom(0.1))

#model.add(Conv2D(32, (3, 3), activation='elu', padding='valid' , input_shape=(None, None, 3)))
model.add(Conv2D(32, (3, 3), activation='elu', padding='valid')) # , input_shape=(None, None, 3)
model.add(MaxPooling2D((2, 2), padding='valid'))
model.add(Conv2D(32, (3, 3), activation='elu', padding='valid'))
model.add(MaxPooling2D((2, 2), padding='valid'))
model.add(Conv2D(32, (3, 3), activation='elu', padding='valid'))
model.add(MaxPooling2D((2, 2), padding='valid'))
model.add(Conv2D(32, (3, 3), activation='elu', padding='valid'))
model.add(MaxPooling2D((2, 2), padding='valid'))
model.add(Conv2D(32, (3, 3), activation='elu', padding='valid'))
model.add(MaxPooling2D((2, 2), padding='valid'))

# Comment if using Dense
# model.add(Conv2D(filters=5, kernel_size=1, strides=1))
# model.add(GlobalAveragePooling2D())  # Add global pooling layer
# model.add(Activation('softmax'))

#UnComment if using Dense
model.add(GlobalAveragePooling2D())  # Add global pooling layer
model.add(Dense(5, activation='softmax'))  # Use a Dense layer for classification

model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=[macro_f1])
es = EarlyStopping(monitor='macro_f1', mode='max', verbose=1) #val_macro_f1

#Train the model
model.fit(train_generator, epochs=20, steps_per_epoch = len(train_generator), callbacks=[es]) #, validation_data=test_generator

Epoch 1/20
189/189 [==============================] - 60s 231ms/step - loss: 1.1815 - macro_f1: 0.4955
Epoch 2/20
189/189 [==============================] - 31s 165ms/step - loss: 1.1615 - macro_f1: 0.4957
Epoch 3/20
189/189 [==============================] - 24s 129ms/step - loss: 1.1488 - macro_f1: 0.4953
Epoch 3: early stopping


In [ ]:
train_loss, train_fscore = model.evaluate(train_generator)
print(train_fscore)
test_loss, test_fscore = model.evaluate(test_generator)
print(test_fscore)

189/189 [==============================] - 20s 107ms/step - loss: 1.1470 - macro_f1: 0.4953
0.49534571170806885
74/74 [==============================] - 9s 121ms/step - loss: 1.4759 - macro_f1: 0.4830
0.48303407430648804


In [ ]:
model.save("model", include_optimizer=False, save_format='tf')

In [ ]:
!zip -r 'model.zip' '/content/model'

  adding: content/model/ (stored 0%)
  adding: content/model/assets/ (stored 0%)
  adding: content/model/keras_metadata.pb (deflated 94%)
  adding: content/model/saved_model.pb (deflated 89%)
  adding: content/model/variables/ (stored 0%)
  adding: content/model/variables/variables.data-00000-of-00001 (deflated 14%)
  adding: content/model/variables/variables.index (deflated 63%)
  adding: content/model/fingerprint.pb (stored 0%)
